# Development and Training of a Sinhala (si) Named Entity Recognition (NER) Model for SpaCy Framework

## Ashen Weligalle (thawe276)

In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

si_nlp= spacy.blank("si") # loading si model
db= DocBin()

# Open and load the JSON file with the correct encoding
with open('./data/si_core_web_md_train.json', encoding='utf-8') as jsont_dat:
    train_data = json.load(jsont_dat)

with open('./data/si_core_web_md_valid.json', encoding='utf-8') as jsonv_dat:
    valid_data = json.load(jsonv_dat)

Train and valid data open as a JSON format. In the below code snipt convert and save both, train and valid data as a SpaCy object. If there is a span empty with data it indicate as "skipping entity".

In [2]:
# Crating spacy object using train_data and valid_data
for text, annotations in tqdm(train_data['annotations']):
    doc = si_nlp.make_doc(text)
    ents=[]
    for start,end, label in annotations['entities']:
        span= doc.char_span(start,end,label=label,alignment_mode='contract')
        if span is None:
            print('Skipping entity')
        else:
            ents.append(span)
    doc.ents= ents
    db.add(doc)

db.to_disk("./data/si_core_web_md_train.spacy")

for text, annotations in tqdm(valid_data['annotations']):
    doc = si_nlp.make_doc(text)
    ents=[]
    for start,end, label in annotations['entities']:
        span= doc.char_span(start,end,label=label,alignment_mode='contract')
        if span is None:
            print('Skipping entity')
        else:
            ents.append(span)
    doc.ents= ents
    db.add(doc)

db.to_disk("./data/si_core_web_md_valid.spacy")

100%|██████████| 357/357 [00:00<00:00, 1497.93it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 15/15 [00:00<00:00, 6554.28it/s]


In [19]:
! python -m spacy init config config.cfg --lang si --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: si
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [26]:
!python -m spacy train config.cfg --output ./model --paths.train ./data/si_core_web_md_train.spacy --paths.dev ./data/si_core_web_md_train.spacy

ℹ Saving to output directory: model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     51.41    5.03    3.28   10.83    0.05
  2     200       1199.69   6874.78   33.32   44.12   26.77    0.33
  5     400       1610.90   5409.58   66.81   68.54   65.17    0.67
  9     600       1489.28   3802.99   89.45   89.67   89.23    0.89
 14     800       1130.51   2118.96   95.94   95.88   96.00    0.96
 19    1000        981.34   1380.79   98.55   98.76   98.34    0.99
 26    1200        861.88   1051.06   98.34   98.52   98.15    0.98
 35    1400       1035.63    937.87   99.11   99.08   99.14    0.99
 45    1600        984.82    842

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [32]:
si_ner_nlp=spacy.load("./model/model-best")

In [33]:
tex=si_ner_nlp(''' ජනපති හමුවීමට නියමිත චීන ප්‍රධානින් කවුද ?
මෙම සංචාරයේ දී චීන ජනාධිපති ෂී ජින් පිං සමග අනුර කුමාර දිසානායක ජනාධිපතිවරයා, අන්‍යොන්‍ය වශයෙන් උනන්දුවක් දක්වන ක්ෂේත්‍ර ගණනාවක් පිළිබඳව සාකච්ඡා පැවැත්වීමට සැලසුම් කර ඇති බව ශ්‍රී ලංකාවේ විදේශ කටයුතු අමාත්‍යංශය නිවේදනයක් නිකුත් කරමින් පවසයි.

ජනාධිපති අනුර කුමාර දිසානායක මෙම සංචාරයේදී චීන අග්‍රාමාත්‍ය ලී ච්‍යං (Li Qiang) සහ චීන ජාතික මහජන කොංග්‍රසයේ ස්ථාවර කමිටුවේ සභාපති ජාඕ ලීජි ඇතුළු රාජ්‍ය තාන්ත්‍රිකයින් පිරිසක් ද හමුවීමට නියමිත යි.

එමෙන්ම මෙම චීන සංචාරයේදී ජනාධිපති අනුර කුමාර දිසානායක, තාක්ෂණික හා කෘෂිකාර්මික සංවර්ධනය මෙන්ම දිළිඳුකම පිටුදැකීම ආදී අරමුණු පෙරටු කරගත් ක්ෂේත්‍ර චාරිකා කිහිපයකට සහභාගි වීමට නියමිත අතර ඉහළ පෙළේ ව්‍යාපාරික හමු කිහිපයකට එක්වීමට ද සැලසුම් කර තිබේ.

චීන කොමියුනිස්ට් පක්ෂය සමග ඇති සබඳතා මෙන්ම චීන ආයෝජකයින්ට සිත් ඇදගන්නා සුළු ආයෝජන අවස්ථා ලබාදීම හරහා ජනාධිපති අනුර කුමාර දිසානායකට චීනය සමග හොඳ ප්‍රතිරූපයක් ගොඩනගා ගැනීමට හැකි වනු ඇති බව ජාත්‍යන්තර සබඳතා පර්යේෂක සහ උපදේශක ආචාර්ය හසිත් කන්දඋඩහේවා බීබීසී සිංහල සේවයට අදහස් දක්වමින් පැවසීය.''')

In [37]:
spacy.displacy.render(tex,style="ent",jupyter=True)